In [2]:
import keras
import numpy as np
import pandas as pd
from hyperopt import STATUS_OK, Trials,fmin, hp,tpe
from sklearn.metrics import mean_squared_error
from sklearn. model_selection import train_test_split

In [3]:
import mlflow
from mlflow.models import infer_signature

In [4]:
df = pd.read_csv(
    "https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv",
    sep=";",
)

In [6]:
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [7]:
train , test = train_test_split(df,test_size=0.2,random_state=42)

In [8]:
train_x = train.drop(['quality'],axis=1).values
train_y = train['quality'].values.ravel()

In [9]:
test_x = test.drop(['quality'],axis=1).values
test_y = test['quality'].values.ravel()

In [10]:
train_x,valid_x, train_y, valid_y = train_test_split(train_x, train_y, test_size=0.2, random_state=42)

In [11]:
signature = infer_signature(train_x, train_y)

In [13]:
def train_model(params, epochs, train_x, train_y, valid_x, valid_y, test_x, test_y):
    mean = np.mean(train_x, axis=0)
    var = np.var(train_x, axis=0)
    
    model = keras.Sequential(
        [
            keras.Input([train_x.shape[1]]),
            keras.layers.Normalization(mean=mean, variance=var),
            keras.layers.Dense(64, activation='relu'),
            keras.layers.Dense(1),
        ]
    )
    
    model.compile(optimizer=keras.optimizers.SGD(
        learning_rate=params["lr"],
        momentum=params["momentum"]
    ),
    loss="mean_squared_error",
    metrics=[keras.metrics.RootMeanSquaredError()]              
    )
    
    with mlflow.start_run(nested=True):
        model.fit(train_x, train_y,validation_data=(valid_x, valid_y), epochs= epochs, batch_size=64)
        eval_result = model.evaluate(test_x, test_y, batch_size=64)
        eval_rmse = eval_result[1]
        
        mlflow.log_params(params)
        mlflow.log_metric("eval_rmse",eval_rmse)
        
        mlflow.tensorflow.log_model(model,"model", signature=signature)
        
        return {"loss": eval_rmse, "status": STATUS_OK, "model": model}

In [14]:
def objective(params):
    results = train_model(
        params,
        epochs=3,
        train_x=train_x,
        train_y=train_y,
        valid_x=valid_x,
        valid_y=valid_y,
        test_x=test_x,
        test_y=test_y,
    )
    return results

In [18]:
from hyperopt import hp
import numpy as np

space = {
    "lr": hp.loguniform("lr", np.log(1e-5), np.log(1e-1)),  # Learning rate in log scale
    "momentum": hp.uniform("momentum", 0.0, 1.0)           # Momentum strictly between 0 and 1
}


In [19]:
mlflow.set_tracking_uri(uri='http://127.0.0.1:5000')
mlflow.set_experiment("Winequality")
with mlflow.start_run():
    # Conting hyper search using hyper opts
    trials = Trials()
    best  = fmin(
        fn=objective,
        space=space,
        algo=tpe.suggest,
        max_evals=4,
        trials=trials
    )
    
    best_run = sorted(trials.results, key= lambda x: x["loss"])[0]
    
    
    mlflow.log_params(best)
    mlflow.log_metric("eval_rmse",best_run["loss"])
    mlflow.tensorflow.log_model(best_run["model"],"model", signature=signature)
    
    
    print(f"best parameters: {best}")
    print(f"Best eval rmse: {best_run['loss']}")

Epoch 1/3                                            

 1/49 ━━━━━━━━━━━━━━━━━━━━ 12s 261ms/step - loss: 28.9634 - root_mean_squared_error: 5.3818
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 8.8254 - root_mean_squared_error: 2.8180 - val_loss: 1.4902 - val_root_mean_squared_error: 1.2207

Epoch 2/3                                            

 1/49 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.8716 - root_mean_squared_error: 0.9336
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.0504 - root_mean_squared_error: 1.0247 - val_loss: 1.0277 - val_root_mean_squared_error: 1.0138

Epoch 3/3                                            

 1/49 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.9860 - root_mean_squared_error: 0.9930
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.8860 - root_mean_squared_error: 0.9405 - val_loss: 0.8114 - val_root_mean_squared_error: 0.9008

 1/16 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.6812 - root_mean_squared_error: 0.8253
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step